
## Use case - this is an adaptation from Langchain example notebook

SQL Chain and SQL agent are two ways we can use natural language to query our database. This needs you to have connectivity with the database and access to an LLM, python runtime.



### Setup
First, you'll need to have the langchain library installed, along with its dependencies. Since we're using the SQLDatabase chain, we'll install that as well. Since this is an experimental lib, we'll need to include `langchain_experimental` in our installs. We'll then import the necessary modules.

But first let us setup a sample database: Chinook is the choice of database and load it with sample data.

In [6]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('Chinook.db')

# Read SQL from file
with open('Chinook_Sqlite.sql', 'r') as sql_file:
    sql_script = sql_file.read()

# Execute the SQL script
cur = conn.cursor()
cur.executescript(sql_script)

# Test query
cur.execute("SELECT * FROM Artist LIMIT 10;")
rows = cur.fetchall()

# Print results
for row in rows:
    print(row)

# Close the connection
conn.close()


(1, 'AC/DC')
(2, 'Accept')
(3, 'Aerosmith')
(4, 'Alanis Morissette')
(5, 'Alice In Chains')
(6, 'Antônio Carlos Jobim')
(7, 'Apocalyptica')
(8, 'Audioslave')
(9, 'BackBeat')
(10, 'Billy Cobham')


In [7]:
!pip install -U langchain langchain_experimental openai
# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

from langchain.llms import OpenAI
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

import os
from google.colab import userdata


os.environ['OPENAI_API_KEY'] = userdata.get('openai_api_key')

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
llm = OpenAI(temperature=0, verbose=True)





2. Create the SQL Query using SQL chain and get the results

In [8]:

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

db_chain.run("How many employees are there?")





> Entering new SQLDatabaseChain chain...
How many employees are there?
SQLQuery:SELECT COUNT(*) FROM "Employee";
SQLResult: [(8,)]
Answer:There are 8 employees.
> Finished chain.


'There are 8 employees.'

3. Create SQL query itself

In [9]:
from langchain.chains import create_sql_query_chain
from langchain.chat_models import ChatOpenAI


chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
response = chain.invoke({"question": "How many employees are there"})
print(response)



SELECT COUNT(*) FROM Employee


4. Execute the response on the DB

In [10]:
db.run(response)


'[(8,)]'

5. Create the SQL Agent

In [11]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit

# from langchain.agents import AgentExecutor
from langchain.agents.agent_types import AgentType

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0)),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

6. Run a complex query via an agent

In [12]:
agent_executor.run(
    "List the total sales per country. Which country's customers spent the most?"
)




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought: I should query the schema of the Invoice and Customer tables.
Action: sql_db_schema
Action Input: Invoice, Customer
Observation: 
CREATE TABLE "Customer" (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES "Employee" ("EmployeeId")
)

/*
3 rows from Customer table:
CustomerId	FirstName	LastName	Company	Address	City	State	Country	PostalCode	Phone	Fax	Email	SupportRepId
1	Luís	Gonçalves	Embraer - Empresa

'The customer from the USA spent the most, with a total of $523.06.'